In [3]:
# Install dependencies
!pip uninstall -y tensorflow
!pip install transformers

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 19.5MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b95db337e947cbed07370121f1c4d4ddf1d0f7e29a2d907304e7468454fe2072
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import json
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import transformers
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertModel, BertTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

In [6]:
class SentimentClassifier(nn.Module):
  """
  This class defines the model architecture which is simply a fully-connected
  layer on top of a pre-trained BERT model. 
  """

  def __init__(self, BERT_MODEL):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3) # Number of output classes = 3

  def forward(self, ids, mask, token_type_ids):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
    output = self.drop(pooled_output)
    return self.out(output)

# Inference on BERT-single Testing Set

The best trained model (based on validation set) is utilized to perform inference on testing sets corresponding to every `location-aspect`. 

The predicted values are stored to perform evaluation. 

In [ ]:
class SentiHood:
  """
  This class tokenizes the input text using the pre-trained BERT tokenizer 
  (wordpiece) and returns the corresponding tensors.
  """
  
  def __init__(self, opinions_id, text, targets, tokenizer, max_len):
    self.opinions_id = opinions_id
    self.text = text
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, item):
    opinions_id = self.opinions_id[item]
    text = str(self.text[item])
    targets = self.targets[item]

    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = self.max_len,
        pad_to_max_length = True
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        "ids": torch.tensor(ids, dtype=torch.long),
        "mask": torch.tensor(mask, dtype=torch.long),
        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        "targets": torch.tensor(targets, dtype=torch.long),
        "opinions_id": torch.tensor(opinions_id, dtype=torch.long)
    }

In [ ]:
def infer_loop_function(data_loader, model, device, location, aspect):
  """
  This function performs the inference on testing sets and stores the predicted
  values.
  """

  model.eval()

  df_pred = pd.DataFrame({"id": [], "predicted": [], "actual": []})

  ii = 0
  for bi, d in tqdm(enumerate(data_loader), total=len(data_loader), ncols=80, leave=False):
    opinions_id = d["opinions_id"]
    ids = d["ids"]
    mask = d["mask"]
    token_type_ids = d["token_type_ids"]
    targets = d["targets"]

    opinions_id = opinions_id.to(device, dtype=torch.long)
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.long)

    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
    _, predicted = torch.max(outputs, 1)
    
    predicted = predicted.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    opinions_id = opinions_id.detach().cpu().numpy()

    for k in range(len(predicted)):
      df_pred.loc[ii] = [str(opinions_id[k]), str(predicted[k]), str(targets[k])]
      ii += 1

  print(f"{location}{aspect} DONE!")
  save_path = '/content/drive/My Drive/SentiHood/Bert-single/PredictedData/Predicted' + str(location) + str(aspect) + '.csv'
  df_pred.to_csv(save_path, index=False)


In [ ]:
def run():
  """
  This function defines the necessary hyperparameters and models. It also 
  loads and tokenizes the testing dataset and execute the inference procedure.
  """
  
  MAX_LEN = 140
  BATCH_SIZE = 16
  BERT_MODEL = 'bert-base-uncased'

  locations = ['LOCATION1', 'LOCATION2']
  aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

  for location in locations:
    for aspect in aspects:
      print(f"Starting {location} {aspect}...")
      testing_set_path = '/content/drive/My Drive/SentiHood/Bert-single/TestingData/' + str(location) + str(aspect) + '.csv'

      df_test = pd.read_csv(testing_set_path)
      sentiment_mapping = {
          'Positive': 0,
          'Negative': 1,
          'None': 2
      }
      df_test['sentiment'] = df_test['sentiment'].map(sentiment_mapping)
      df_test = df_test.reset_index(drop=True)

      tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)

      test_dataset = SentiHood(
          opinions_id = df_test['id'].values,
          text = df_test['text'].values,
          targets = df_test['sentiment'].values,
          tokenizer = tokenizer,
          max_len = MAX_LEN
      )

      test_data_loader = torch.utils.data.DataLoader(
          test_dataset,
          batch_size = BATCH_SIZE,
          shuffle = False
      )

      device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      print(f"Device: {device}")

      model = torch.load('/content/drive/My Drive/SentiHood/Bert-single/LocationAspectModels/'+str(location)+str(aspect)+'/best.bin')
      infer_loop_function(data_loader=test_data_loader, model=model, device=device, location=location, aspect=aspect)

if __name__ == "__main__":
  run()     

Starting LOCATION1 dining...



Device: cuda:0


LOCATION1dining DONE!
Starting LOCATION1 general...
Device: cuda:0


 47%|████████████████████▏                      | 44/94 [00:06<00:07,  7.12it/s]

# BERT-single Evaluation

### Creating a dataframe containing true labels corresponding to every `location-aspect` in the testing set.

In [7]:
df_true_location1 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/Bert-single/TestingData/LOCATION1' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  sentiment_mapping = {
      'Positive': 0,
      'Negative': 1,
      'None': 2
  }
  df_test['sentiment'] = df_test['sentiment'].map(sentiment_mapping)
  df_test = df_test.reset_index(drop=True)

  df_true_location1[aspect] = df_test['sentiment']

df_true_location1['location'] = 'LOCATION1'
df_true_location1['id'] = df_test['id']

In [8]:
df_true_location2 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/Bert-single/TestingData/LOCATION2' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  sentiment_mapping = {
      'Positive': 0,
      'Negative': 1,
      'None': 2
  }
  df_test['sentiment'] = df_test['sentiment'].map(sentiment_mapping)
  df_test = df_test.reset_index(drop=True)

  df_true_location2[aspect] = df_test['sentiment']

df_true_location2['location'] = 'LOCATION2'
df_true_location2['id'] = df_test['id']

In [9]:
df_true = pd.concat([df_true_location1, df_true_location2])
df_true

,id,location,dining,general,green-nature,live,multicultural,nightlife,price,quiet,safety,shopping,touristy,transit-location
0,153,LOCATION1,2,2,2,2,2,2,2,2,0,2,2,2
1,1130,LOCATION1,2,0,2,2,2,2,2,2,0,2,2,2
2,1271,LOCATION1,2,1,2,2,2,2,0,2,2,2,2,2
3,1089,LOCATION1,2,1,2,2,2,2,2,2,2,2,2,2
4,731,LOCATION1,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,1431,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2
384,1290,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2
385,363,LOCATION2,2,2,2,2,2,2,2,2,2,2,2,2
386,1304,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2


### Creating dataframe containing predicted labels corresponding to every `location-aspect` in the testing set.

In [10]:
df_predicted_location1 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/Bert-single/PredictedData/PredictedLOCATION1' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  df_test = df_test.reset_index(drop=True)

  df_predicted_location1[aspect] = df_test['predicted']

df_predicted_location1['location'] = 'LOCATION1'
df_predicted_location1['id'] = df_test['id']

In [11]:
df_predicted_location2 = pd.DataFrame({'id': [], 'location': [] , 'dining': [], 'general': [], 'green-nature': [], 'live': [], 'multicultural': [], 'nightlife': [], 'price': [], 'quiet': [], 'safety': [],'shopping': [], 'touristy': [], 'transit-location': []})
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']

for aspect in aspects:
  testing_set_path = '/content/drive/My Drive/SentiHood/Bert-single/PredictedData/PredictedLOCATION2' + str(aspect) + '.csv'

  df_test = pd.read_csv(testing_set_path)
  df_test = df_test.reset_index(drop=True)

  df_predicted_location2[aspect] = df_test['predicted']

df_predicted_location2['location'] = 'LOCATION2'
df_predicted_location2['id'] = df_test['id']

In [12]:
df_predicted = pd.concat([df_predicted_location1, df_predicted_location2])
df_predicted

,id,location,dining,general,green-nature,live,multicultural,nightlife,price,quiet,safety,shopping,touristy,transit-location
0,153,LOCATION1,2,2,2,2,2,2,2,2,0,2,2,2
1,1130,LOCATION1,2,0,2,2,2,2,2,2,0,2,2,2
2,1271,LOCATION1,2,1,2,2,2,2,1,2,2,2,2,2
3,1089,LOCATION1,2,0,2,2,2,2,2,2,2,2,2,2
4,731,LOCATION1,2,2,2,2,2,2,2,2,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,1431,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2
384,1290,LOCATION2,2,0,2,2,2,2,2,2,2,2,2,2
385,363,LOCATION2,2,2,2,2,2,2,2,2,2,2,2,2
386,1304,LOCATION2,2,2,2,2,2,2,1,2,2,2,2,2


## Evaluation

### Sentiment Accuracy

In [ ]:
def compute_sentiment_accuracy(df_true, df_predicted):
  """This function computes the sentiment classfication accuracy"""

  count = 0
  total = 0

  for aspect in ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']:
    count += np.sum(df_true[aspect].values == df_predicted[aspect].values)
    total += df_true.shape[0]

  accuracy = float(count)/float(total) * 100
  return round(accuracy, 2)

In [ ]:
print(f"Sentiment Accuracy: {compute_sentiment_accuracy(df_true, df_predicted)}")

Sentiment Accuracy: 96.45


### Aspect Accuracy

In [ ]:
def compute_aspect_accuracy(df_true, df_predicted):
  """
  This function computes the strict aspect accuracy.
  0 => Represents that the aspect has not been detected by the model.
  1 => Represents that the aspect has been detected by the model.
  """
  
  df_true = df_true.replace([0, 1], 1).replace(2, 0)
  df_predicted = df_predicted.replace([0, 1], 1).replace(2, 0)

  count = 0
  total = 0

  for i in range(df_true.shape[0]):
    true_values = df_true.iloc[i].values[2:]
    predicted_values = df_predicted.iloc[i].values[2:]

    if (true_values == predicted_values).all():
      count += 1
    total += 1

  accuracy = float(count)/float(total)*100
  return round(accuracy, 2)

In [ ]:
print(f"Aspect Accuracy (strict): {compute_aspect_accuracy(df_true, df_predicted)}%")

Aspect Accuracy (strict): 68.6%


### Aspect F1 Score

In [ ]:
def compute_aspect_f1_score(df_true, df_predicted):
  """
  This function computest the macro F1 score of predicted aspects.
  0 => Represents that the aspect has not been detected by the model.
  1 => Represents that the aspect has been detected by the model.
  """

  df_true = df_true.replace([0, 1], 1).replace(2, 0)
  df_predicted = df_predicted.replace([0, 1], 1).replace(2, 0)

  total_f1_score = 0
  total = 0

  for i in range(df_true.shape[0]):
    true_values = list(df_true.iloc[i].values[2:])
    predicted_values = list(df_predicted.iloc[i].values[2:])

    total_f1_score += f1_score(true_values, predicted_values, average="macro")
    total += 1

  score = float(total_f1_score)/float(total)*100
  return round(score, 2)

In [ ]:
print(f"Aspect F1 score: {compute_aspect_f1_score(df_true, df_predicted)}")

Aspect F1 score: 88.19


# Prediction Result Analysis

This section analyses the predicted results to find the aspects and sentiments that are most and least accurate. It utilizes the `df_true` and `df_predicted` dataframes constructed in the **Evaluation** section.

In [16]:
"""
Computes the positive_correct, positive_total, negative_correct, negative_total, 
none_correct, none_total corresponding to all the aspects of LOCATION1 and 
LOCATION2.
"""

locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']
df_location_aspect = pd.DataFrame({"location": [], "aspect": [], "positive correct": [], "positive total": [], "negative correct": [], "negative total": [], "none correct": [], "none total": [],})

ii = 0
for location in locations:
  for aspect in aspects:
    positive_total = df_true[(df_true[aspect] == 0) & (df_true['location'] == location)].shape[0]
    negative_total = df_true[(df_true[aspect] == 1) & (df_true['location'] == location)].shape[0]
    none_total = df_true[(df_true[aspect] == 2) & (df_true['location'] == location)].shape[0]

    positive_correct = 0
    for i in df_true[(df_true[aspect] == 0) & (df_true['location'] == location)].index:
      if df_predicted.iloc[i][aspect] == df_true.iloc[i][aspect]:
        positive_correct += 1

    negative_correct = 0
    for i in df_true[(df_true[aspect] == 1) & (df_true['location'] == location)].index:
      if df_predicted.iloc[i][aspect] == df_true.iloc[i][aspect]:
        negative_correct += 1

    none_correct = 0
    for i in df_true[(df_true[aspect] == 2) & (df_true['location'] == location)].index:
      if df_predicted.iloc[i][aspect] == df_true.iloc[i][aspect]:
        none_correct += 1

    df_location_aspect.loc[ii] = [location, aspect, positive_correct, positive_total, negative_correct, negative_total, none_correct, none_total]
    ii += 1

In [17]:
df_location_aspect['positive percentage'] = round(df_location_aspect['positive correct']/df_location_aspect['positive total']*100, 2)
df_location_aspect['negative percentage'] = round(df_location_aspect['negative correct']/df_location_aspect['negative total']*100, 2)
df_location_aspect['none percentage'] = round(df_location_aspect['none correct']/df_location_aspect['none total']*100, 2)

df_location_aspect['total percentage'] = round((df_location_aspect['positive correct'] + df_location_aspect['negative correct'] + df_location_aspect['none correct'])/(df_location_aspect['positive total'] + df_location_aspect['negative total'] + df_location_aspect['none total'])*100, 2)

In [18]:
df_location_aspect

,location,aspect,positive correct,positive total,negative correct,negative total,none correct,none total,positive percentage,negative percentage,none percentage,total percentage
0,LOCATION1,dining,31.0,31.0,0.0,2.0,1445.0,1458.0,100.00,0.00,99.11,98.99
1,LOCATION1,general,317.0,359.0,77.0,113.0,904.0,1019.0,88.30,68.14,88.71,87.06
2,LOCATION1,green-nature,33.0,40.0,0.0,0.0,1445.0,1451.0,82.50,NaN,99.59,99.13
3,LOCATION1,live,48.0,64.0,11.0,23.0,1382.0,1404.0,75.00,47.83,98.43,96.65
4,LOCATION1,multicultural,30.0,39.0,1.0,3.0,1441.0,1449.0,76.92,33.33,99.45,98.73
5,LOCATION1,nightlife,57.0,63.0,0.0,2.0,1416.0,1426.0,90.48,0.00,99.30,98.79
6,LOCATION1,price,65.0,81.0,99.0,116.0,1271.0,1294.0,80.25,85.34,98.22,96.24
7,LOCATION1,quiet,11.0,14.0,5.0,15.0,1462.0,1462.0,78.57,33.33,100.00,99.13
8,LOCATION1,safety,54.0,61.0,48.0,66.0,1331.0,1364.0,88.52,72.73,97.58,96.11
9,LOCATION1,shopping,61.0,62.0,0.0,1.0,1415.0,1428.0,98.39,0.00,99.09,98.99


# Creating preds.jsonl

This section constructs the `preds.jsonl` file which contains model predictions and original annotations in the following json format.


```
{
  "opinions": [
    {
      "sentiment": "Positive",
      "aspect": "safety",
      "target_entity": "LOCATION1"
    }
  ],
  "id": 153,
  "text": " LOCATION1 is in Greater London and is a very safe place",
  "model_pred": [
    {
      "sentiment": ...,
      "aspect": ...,
      "target_entity":...
    },...
  ]
}
```

In [ ]:
with open('/content/drive/My Drive/SentiHood/SentiHood Dataset/sentihood-test.json', 'r') as fp:
  testing_set = json.load(fp)

In [ ]:
labels_to_sentiment_dict = {
    0: 'Positive',
    1: 'Negative',
    2: 'None'
}

In [ ]:
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

# models_dict:Is a dictionary containing models corresponding to all the 
# `location-aspect`.

models_dict = {}

for location in locations:
  for aspect in tqdm(aspects, ncols=80):
    model = torch.load('/content/drive/My Drive/SentiHood/Bert-single/LocationAspectModels/'+str(location)+str(aspect)+'/best.bin')
    models_dict[f"{location}{aspect}"] = model

100%|███████████████████████████████████████████| 12/12 [02:09<00:00, 10.78s/it]


In [ ]:
BERT_MODEL = 'bert-base-uncased'
MAX_LEN = 160

tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

for each_example in tqdm(testing_set, ncols=80):
  id = each_example['id']
  text = each_example['text'].strip()

  each_example['model_pred'] = []

  for location in locations:
    if location in text:
      # If "location" is present in the text, then utilize the trained models
      # to predict the aspects and their corresponding sentiment of the text.

      for aspect in aspects:
        inputs = tokenizer.encode_plus(
            text,
            add_special_tokens = True,
            max_length = MAX_LEN,
            pad_to_max_length = True
        )
        ids = torch.tensor(inputs["input_ids"], dtype=torch.long).unsqueeze(0)
        mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).unsqueeze(0)
        token_type_ids = torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze(0)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        model = models_dict[f"{location}{aspect}"]
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        _, predicted = torch.max(outputs, 1)

        predicted = predicted.detach().cpu().numpy()

        # If predicted sentiment is not None, then add it to the preds.jsonl.

        if predicted[0] != 2:
          result = {
              "sentiment": labels_to_sentiment_dict[predicted[0]],
              "aspect": aspect,
              "target_entity": location
          }
          each_example['model_pred'].append(result)

  0%|                                                  | 0/1491 [00:00<?, ?it/s]


Device: cuda:0


100%|███████████████████████████████████████| 1491/1491 [14:02<00:00,  1.77it/s]


In [ ]:
with open('/content/drive/My Drive/SentiHood/Bert-single/preds.jsonl', mode='w', encoding='utf-8') as fp:
  for each in testing_set:
    json_record = json.dumps(each, ensure_ascii=False)
    fp.write(json_record + '\n')